Method 1. From https://arxiv.org/pdf/1803.06987 

In [ ]:

def symplectic_inner_product(u, v):
    n = len(u) // 2
    return np.dot(u[:n], v[n:]) + np.dot(u[n:], v[:n]) % 2

def fftshift(x):
    """Cyclically shifts vector x (used to match MATLAB fftshift for vectors)."""
    n = len(x)
    return np.roll(x, n // 2)

def gf2_solve(A, b):
    """Solve Ax = b over GF(2)."""
    from scipy.linalg import lu
    A = A % 2
    b = b % 2
    m, n = A.shape
    aug = np.hstack([A, b.reshape(-1,1)]) % 2
    # Row reduce manually over GF(2)
    for col in range(n):
        for row in range(col, m):
            if aug[row, col] == 1:
                aug[[col, row]] = aug[[row, col]]
                break
        else:
            continue  # No pivot in this column
        for row in range(m):
            if row != col and aug[row, col] == 1:
                aug[row] ^= aug[col]
    x = aug[:, -1][:n]
    return x

def find_symp_mat(X, Y):
    m, dim = X.shape
    n = dim // 2
    F = np.eye(2 * n, dtype=int)

    def Z_h(h):
        return (np.eye(2*n, dtype=int) + np.outer(fftshift(h), h)) % 2

    def find_w(x, y, Ys):
        A = np.vstack([x, y, Ys])
        A = fftshift(A)
        b = np.concatenate([[1], [1], [symplectic_inner_product(Ys[i], y) for i in range(len(Ys))]])
        return gf2_solve(A, b)

    for i in range(m):
        x_i = X[i]
        y_i = Y[i]
        x_it = x_i @ F % 2
        if np.array_equal(x_it, y_i):
            continue
        if symplectic_inner_product(x_it, y_i) == 1:
            h_i = (x_it + y_i) % 2
            F = (F @ Z_h(h_i)) % 2
        else:
            w_i = find_w(x_it, y_i, Y[:i])
            h_i1 = (w_i + y_i) % 2
            h_i2 = (x_it + w_i) % 2
            F = (F @ Z_h(h_i1)) % 2
            F = (F @ Z_h(h_i2)) % 2
    return F


Method 2

In [1]:
import numpy as np
from sympy import Matrix

def is_symplectic(M, d):
    n = M.shape[0] // 2
    if M.shape[0] != M.shape[1] or M.shape[0] % 2 != 0:
        return False
    J = np.block([[np.zeros((n, n), dtype=int), np.eye(n, dtype=int)],
                  [-np.eye(n, dtype=int), np.zeros((n, n), dtype=int)]]) % d
    return np.array_equal((M.T @ J @ M) % d, J % d)

def symplectic_form(n, d):
    J = np.block([[np.zeros((n, n), dtype=int), np.eye(n, dtype=int)],
                  [-np.eye(n, dtype=int), np.zeros((n, n), dtype=int)]]) % d
    return J

def gram_schmidt_symplectic(X, d):
    from sympy import Matrix

    X = Matrix(X.tolist())
    X = X.applyfunc(lambda x: x % d)

    n = X.rows
    m = X.cols

    basis = []
    for i in range(m):
        x = X.col(i)
        if x.norm() == 0:
            continue
        for b in basis:
            x -= x.dot(b) * b
            x = x.applyfunc(lambda xi: xi % d)
        if x.norm() == 0:
            continue
        basis.append(x)

    if not basis:
        return np.zeros((n, 0), dtype=int)
    B = Matrix.hstack(*basis)
    return np.array(B.tolist(), dtype=int) % d

def find_symp_mat(X, Y, d):
    X = np.array(X, dtype=int) % d
    Y = np.array(Y, dtype=int) % d

    assert X.shape == Y.shape
    n2, k = X.shape
    assert n2 % 2 == 0
    n = n2 // 2

    J = symplectic_form(n, d)
    X_sym = gram_schmidt_symplectic(X, d)
    Y_sym = gram_schmidt_symplectic(Y, d)

    try:
        Xinv = Matrix(X_sym.tolist()).inv_mod(d)
    except:
        raise ValueError("X_sym is not invertible mod d")

    F = (Matrix(Y_sym.tolist()) * Xinv) % d
    F_np = np.array(F.tolist(), dtype=int)
    return F_np
    # if is_symplectic(F_np, d) and np.array_equal((F_np @ X) % d, Y % d):
    #     return F_np
    # else:
    #     raise ValueError("Failed to construct symplectic matrix")
d = 2
X = np.array([
    [1, 0, 1, 1],
    [0, 1, 0, 1],
    [0, 1, 0, 0],
    [1, 0, 0, 1]
], dtype=int)


Y = np.array([
    [1, 1, 0, 1],
    [0, 1, 1, 1],
    [0, 1, 0, 0],
    [1, 1, 1, 1]
], dtype=int)
F = find_symp_mat(X, Y, d)
print("Found symplectic F:")
print(F)
print("Check (F @ X) % d == Y % d:")
print((F @ X) % d)
print(Y)

Found symplectic F:
[[0 1 0 1]
 [1 0 1 1]
 [0 0 1 0]
 [1 0 1 0]]
Check (F @ X) % d == Y % d:
[[1 1 0 0]
 [0 1 1 0]
 [0 1 0 0]
 [1 1 1 1]]
[[1 1 0 1]
 [0 1 1 1]
 [0 1 0 0]
 [1 1 1 1]]


Test both and compare

In [2]:
# === EXAMPLE ===
d = 2

H = np.array([
    [1, 0, 1, 1],
    [0, 1, 0, 1],
    [0, 1, 0, 0],
    [1, 0, 0, 1]
], dtype=int)


H_prime = np.array([
    [1, 1, 0, 1],
    [0, 1, 1, 1],
    [0, 1, 0, 0],
    [1, 1, 1, 1]
], dtype=int)


Ms = find_symplectic_maps(H, H_prime, d, max_solutions=1000)

print(f"Found {len(Ms)} symplectic solutions to H M^T = H' mod {d}:")
for i, M in enumerate(Ms):
    print(f"\nSolution {i+1}:\n{M}")
    print("Check (H @ M.T) % d:")
    print((H @ M.T) % d)

NameError: name 'find_symplectic_maps' is not defined

In [ ]:

def gauss_mod_d(A, b, d):
    A = A.copy() % d
    b = b.copy() % d
    m, n = A.shape
    Ab = np.hstack([A, b.reshape(-1, 1)])

    row = 0
    pivot_cols = []
    for col in range(n):
        pivot_rows = np.where(Ab[row:, col] % d != 0)[0]
        if pivot_rows.size == 0:
            continue
        pivot = pivot_rows[0] + row
        Ab[[row, pivot]] = Ab[[pivot, row]]
        inv = modinv(Ab[row, col], d)
        Ab[row] = (Ab[row] * inv) % d
        for r in range(m):
            if r != row and Ab[r, col] % d != 0:
                Ab[r] = (Ab[r] - Ab[r, col] * Ab[row]) % d
        pivot_cols.append(col)
        row += 1
        if row == m:
            break

    # Extract particular solution (set free variables to 0)
    x = np.zeros(n, dtype=int)
    for i, col in enumerate(pivot_cols):
        x[col] = Ab[i, -1]
    return x